# BERT on GLUE
* Task: Image Classification  
* Dataset: ImageNet  
* Model: ResNet50  

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchvision as tv
import torch.nn.functional as F
import datasets
import evaluate
import sys

sys.path.append('../../')
sys.path.append('../../src/')

import src.general as general
import src.dataset_models as data
import src.metrics as metrics
import src.evaluation as eval
import src.plot as plot
import src.compression.distillation as distill
import src.compression.pruning as prune


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
from transformers import BertForSequenceClassification, BertTokenizerFast, EvalPrediction
from transformers import Trainer, TrainingArguments

# Load the BERT model
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
task = "mrpc"

# Load the GLUE dataset
glue_dataset = datasets.load_dataset("glue" ,task, data_dir='/workspace/volume/data', cache_dir='/workspace/volume/cache')

# Load the glue metric
metric = evaluate.load("glue", task)

Found cached dataset glue (/workspace/volume/cache/glue/mrpc-212aff79ee65f848/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [20]:
from torch.utils.data import DataLoader

kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
train_loader = DataLoader(glue_dataset["train"], batch_size=64, shuffle=True, **kwargs)
val_loader = DataLoader(glue_dataset["validation"], batch_size=64, shuffle=True, **kwargs)
test_loader = DataLoader(glue_dataset["test"], batch_size=64, shuffle=True, **kwargs)

dataset = data.DataSet(name="GLUE", criterion=F.binary_cross_entropy, metric=metric, train_loader=train_loader, test_loader=test_loader)

In [28]:
next(iter(dataset.test_loader)).keys()

dict_keys(['sentence1', 'sentence2', 'label', 'idx'])

In [22]:
general.test(model, dataset)


Test:   0%|          | 0/27 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [ ]:
# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")
    return {"input_ids": inputs.input_ids, "attention_mask": inputs.attention_mask, "labels": examples["label"]}

# Preprocess the datasets
train_dataset = dataset["train"].map(preprocess_function, batched=True)
eval_dataset = dataset["validation"].map(preprocess_function, batched=True)

# Define the Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
)

def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred
    predictions = torch.argmax(logits, dim=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train and evaluate the model
trainer.train()
trainer.evaluate()